## Supervised ML on Descartes Labs Platform:  Interactive Deployment with Dynamic Compute
__________________
This notebook will demonstrate a typical example of how to interact with the results of a supervised machine learning model using Descartes Labs Platform APIs and define new AOIs to analyze on-the-fly.

The general steps covered in this notebook are:
* Retrieve a running [`Function`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function)
* Display results overlain on input imagery in a web map with [`Dynamic Compute`](https://docs.descarteslabs.com/api/dynamic-compute.html)
* Specify new areas to apply our model over with interactive [widgets](https://ipywidgets.readthedocs.io/en/stable/)

_**Note:**_ In order to run this example you must first complete the steps outlined in [02a Generate Training Data.ipynb](02a%20Generate%20Training%20Data.ipynb), [02b Training a Supervised Classifier.ipynb](02b%20Training%20a%20Supervised%20Classifier.ipynb), and [02c Deploying a Supervised Classifier.ipynb](02c%20Deploying%20a%20Supervised%20Classifier.ipynb).

In [ ]:
import yaml

import descarteslabs as dl
import descarteslabs.compute
import descarteslabs.dynamic_compute as dc
import descarteslabs.vector as dl_vector
import geopandas as gpd
import ipyleaflet

In [ ]:
with open("config.yaml", "r") as file:
    config = yaml.load(file, yaml.FullLoader)

In [ ]:
user_hash = dl.auth.Auth().namespace
org = dl.auth.Auth().payload['org']

result_product_id = f"{org or user_hash}:rfc-results-{user_hash}"
result_product_id

## Retrieving an Active Compute Function 

If you lost your ID, you can retrieve it at [app.descarteslabs.com/compute](https://app.descarteslabs.com/compute) or search the latest created Function with that name as below:

In [ ]:
func_search = (
    dl.compute.Function.search()
    .filter(dl.catalog.properties.name.startswith(config["pred_func_name"]))
    .filter(dl.compute.Function.status == dl.compute.FunctionStatus.READY)
    .sort(-dl.compute.Function.creation_date)
    .limit(1)
).collect()

async_func = func_search[0]
async_func.id

## Setting Up Dynamic Compute

Here we will set  up the interactive map components to visualize our study area. 

First we set up a map frame alongside center coordinates and zoom level:

In [ ]:
m = dc.map
m.center = 30.2653, -97.7483
m.zoom = 15

Next create a and visualize a NAIP [`Mosaic`](https://docs.descarteslabs.com/api/dynamic-compute.html#descarteslabs.dynamic_compute.Mosaic) for our time period:

In [ ]:
naip_mosaic = dc.Mosaic.from_product_bands(
    config["product_id"],
    config["bands"],
    start_datetime=config["start"],
    end_datetime=config["end"],
)
naip_mosaic.visualize("FCC", m)

And overlay our input training features as well:

In [ ]:
table = dl_vector.Table.get(config["training_table_name"])
table.visualize("Training Data", m)

Here we define a simple [`DrawControl`](https://ipyleaflet.readthedocs.io/en/latest/controls/draw_control.html) widget which will keep track of drawn polygons:

In [ ]:
# This is some interactivity with the map we'll embed below:
draw_control = ipyleaflet.DrawControl()
# Drawn polygon styling
draw_control.polygon = {
    "shapeOptions": {"fillColor": "green", "color": "blue", "fillOpacity": 0.5},
    "drawError": {"color": "red", "message": "Oops!"},
    "allowIntersection": False,
}

# Setting empty feature collection to track as we draw polys:
feature_collection = {"type": "FeatureCollection", "features": []}

# Define this handle_draw function for the Draw Control widget
def handle_draw(target, action, geo_json):
    # Clears feature collection on each new polygon with new geojson
    feature_collection["features"] = [geo_json]


# Adding the handle_draw function to the Draw Control widget
draw_control.on_draw(handle_draw)
m.add_control(draw_control)

**_Note on Updating Tile Layers_**

Our results mosaic will "refresh" its tile layers upon re-instantiation of it's class, as shown in the cell below. 

If you are waiting for your function to process in real time you will need to re-run the following cell to update your imagery as each job completes:

In [ ]:
rfc_mosaic = dc.Mosaic.from_product_bands(result_product_id, "class")
rfc_mosaic.visualize("RFC Results", m, colormap="terrain")

And display our map:

In [ ]:
m

## Interacting with the Results

Notice that we now have the option embedded in our mapframe to draw new polygons. When you complete a new polygon on the map, run the following cell to format a new list of arguments to pass to the currently running asynchronous function:

In [ ]:
drawn_gdf = gpd.GeoDataFrame.from_features(feature_collection, crs=4326)
# Create a new set of DLTiles for this new AOI
geocontext_geom = drawn_gdf["geometry"][0]
# You could also pass the map's geocontext as a WKT
# geocontext_geom = sgeom.box(*dc.map.geocontext().bounds)
dltiles = dl.geo.DLTile.from_shape(
    geocontext_geom, resolution=config["resolution_m"], tilesize=2048, pad=0
)
args = [(dltile.key, result_product_id) for dltile in dltiles]
len(args)

We can submit those arguments to our running function to process at any time:

In [ ]:
jobs = async_func.map(args)
len(jobs)

## Cleaning Up
It is always best practie to clean up after ourselves, including both Function results and unused Products:

Deleting Function

In [ ]:
async_func.delete_jobs(delete_results=True)
async_func.delete()

Deleting Product

In [ ]:
rfc_prod = dl.catalog.Product.get(result_product_id)
status = rfc_prod.delete_related_objects()
if status:
    status.wait_for_completion()
rfc_prod.delete()